# 1. Import of input data

In [1]:
import nbimporter
import sys
import os
from pathlib import Path

In [2]:
#Import libraries
try:
    current_dir = RUN_DIR
    print(f"📂 ElectricHeatPump.ipynb executed by RUN.ipynb — using directory: {current_dir}")
except NameError:
    current_dir = Path().resolve()
    print(f"📁 ElectricHeatPump.ipynb executed standalone — using directory: {current_dir}")

Libraries_path = (current_dir / "Libraries.ipynb").resolve()

# Se non trovato, prova nei livelli superiori
if not Libraries_path.exists():
    Libraries_path = (current_dir.parent / "Libraries.ipynb").resolve()

# Esegui il file
if Libraries_path.exists():
    print(f"📘 Importing libraries from: {Libraries_path}")
    get_ipython().run_line_magic('run', f'"{Libraries_path}"')
else:
    print("⚠️ Libraries.ipynb not found! Some functions might not be available.")

📁 ElectricHeatPump.ipynb executed standalone — using directory: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\bw\Energy_systems
📘 Importing libraries from: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\bw\Libraries.ipynb
📁 Libraries.ipynb — directory impostata a: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\bw


In [3]:
energysystem = pd.read_excel(energysystem_path, sheet_name = 'Electric Heat Pump')

# 2. Input parameters

In [4]:
Typology = energysystem['Cold source (Groundwater, Surface water, Industrial waste water, Air)']
Power = energysystem['Thermal capacity [kW]']
EnergyProduced = energysystem['Energy produced [kWh /year]']
SCOP = energysystem['SCOP']
ElectricityGrid = energysystem['Energy input from national grid [kWh/year]']
ElectricityPV = energysystem['Energy input from Photovoltaic system [kWh/year]']
ElectricityRICE = energysystem['Energy input from CHP-Internal Combustion Engine [kWh/year]']
ElectricityCombinedCycle = energysystem['Energy input from CHP-Combined cycle [kWh/year]']
ElectricityGasTurbine = energysystem['Energy input from CHP-Gas Turbine [kWh/year]']
ElectricityWtE = energysystem['Energy input from CHP-Waste-to-Energy [kWh/year]']
ElectricityORC = energysystem['Energy input from CHP-Organic Rankine Cycle [kWh/year]']

In [5]:
i = np.ceil(Power / 1000)

conditions = [
    Power < 40,
    (Power >= 40) & (Power <= 1000),
    Power > 1000
]

choices = [
    33.771 * np.exp(0.0758 * Power),
    8.330 * Power + 336.570,
    (8.330 * (Power / i) + 336.570) * i
]

overallmass = np.select(conditions, choices)

Lifespan = 20
GPLifespan = 80

In [6]:
# default parameters
refrigerantgas = np.where(
    Typology.eq('Groundwater') | Typology.eq('Air'),
    np.where((Year >= 2020) & (Year <= 2029), 'R410a', 'R290'),
    np.where(
        Typology.eq('Surface water'),
        np.where((Year >= 2020) & (Year <= 2039), 'R32', 'R290'),
        np.where((Year >= 2020) & (Year <= 2029), 'R410a', 'R290')
    )
)

# 'Well' determination
well = np.where(Typology.eq('Industrial waste water') | Typology.eq('Air'), 'no', 'yes')

# Number of boreholes determination
NW = np.where(
    Typology.eq('Groundwater'), 9 * (35/150),
    np.where(Typology.eq('Surface water'), 2, 0)
)


In [7]:
i = np.ceil(Power / 1000)

conditions = [
    Power < 1000,
    Power >= 1000
]

choices = [
    0.322 * Power**0.932,
    0.322 * (Power / i)**0.932 * i
]

M_R = np.select(conditions, choices)
massrefrigerant = M_R
mass = overallmass - massrefrigerant

# 3. Life cycle inventories

## 3.1 Module A1-3

In [8]:
EHP_reinforcingsteel = 0.58462492250465 * mass
EHP_lowalloyedsteel = 0.155920644761314 * mass
EHP_copper = 0.173796238892333 * mass
EHP_elastomer = 0.0779086588138045 * mass
EHP_PVC = 0.00774953502789833 * mass
EHP_airfun = np.full(len(Power), 1.9)
EHP_electronics = np.full(len(Power), 1)
EHP_waterconsumption = (5.4 * mass) / 1000
EHP_emissionwaterair = 0.15 * EHP_waterconsumption
EHP_emissionwaterwater = 0.02 * mass
EHP_wastewatertreatment = 0.83 * mass
EHP_lubricatingoil = 0.27 * Power
EHP_electricity = 14 * Power
EHP_heat = 136.5 * Power
EHP_refrigerantgas = massrefrigerant
EHP_ethanepentafluoro_methanedifluoro = np.zeros(len(Power)) #0.03 * massrefrigerant
EHP_steelwell = np.where(well == 'Yes', np.full(len(Power), 33), np.zeros(len(Power)))
EHP_PEwell = np.where(well == 'Yes', 4 * Power * 15 * 0.3, np.zeros(len(Power)))
EHP_bentonitewell = np.where(well == 'Yes', np.full(len(Power), 8), np.zeros(len(Power)))
EHP_cementwell = np.where(well == 'Yes', np.full(len(Power), 33), np.zeros(len(Power)))
EHP_glycolethylenewell = np.where(well == 'Yes', 30 * Power * 0.3 * 1.140, np.zeros(len(Power)))
EHP_resourcewaterwell = np.where(well == 'Yes', 1.02 * Power * 1000, np.zeros(len(Power)))
EHP_emissionwaterairwell = np.where(well == 'Yes', (EHP_resourcewaterwell - (30 * Power / 1000)) * 0.15, np.zeros(len(Power)))
EHP_emissionwaterwaterwell = np.where(well == 'Yes', (EHP_resourcewaterwell - (30 * Power / 1000)) * 0.85, np.zeros(len(Power)))
EHP_ironwell = np.where(well == 'Yes', np.full(len(Power), 43), np.zeros(len(Power)))

## 3.2 Module A4

In [9]:
EHP_Truk_16_32 = ((500 + 200)*(overallmass)/1000)
EHP_Light_Truk = (50*(overallmass)/1000)

## 3.3 Module A5

In [10]:
EHP_dieselA5 = np.where(well == 'Yes', np.full(len(Power), 5900), np.zeros(len(Power)))
EHP_Light_TrukA5 = np.where(well == 'Yes', (160 * (EHP_steelwell + EHP_PEwell + EHP_bentonitewell + EHP_cementwell + EHP_glycolethylenewell) / 1000), np.zeros(len(Power)))
EHP_Truk_16_32A5 = np.where(well == 'Yes', (180 * (EHP_steelwell + EHP_PEwell + EHP_bentonitewell + EHP_cementwell + EHP_glycolethylenewell) / 1000), np.zeros(len(Power)))

## 3.4 Module B1

In [11]:
massrefrigerantB2 = np.array(massrefrigerant)
EHP_maintenancerefrigerantgas = np.where(refrigerantgas == 'R290', np.zeros(len(Power)), 0.02 * massrefrigerant * Lifespan)
EHP_maintenanceethanepentafluoro_methanedifluoro = np.where(refrigerantgas == 'R290', np.zeros(len(Power)), 0.02 * massrefrigerant * Lifespan)

## 3.5 Module B2

In [12]:
EHP_Light_Truk_B2 = np.array(50 * (Lifespan / 2) * 0.01 * (massrefrigerantB2 / 1000))  # Light_Truk_B2 come array

## 3.6 Module B4

In [13]:
EHPNumberReplacements = (np.ceil(np.divide(ReqSL, Lifespan) - 1))
SFEHP = (RSP / ReqSL)  # Scaling factor

## 3.7 Module B6

In [14]:
#Parameters for energy input repartition
TotalElectricityInput = (ElectricityGrid + ElectricityPV + ElectricityRICE + ElectricityCombinedCycle +
    ElectricityGasTurbine + ElectricityWtE + ElectricityORC)

EHP_EleGrid = ElectricityGrid / TotalElectricityInput
EHP_ElePV = ElectricityPV / TotalElectricityInput
EHP_EleRICE = ElectricityRICE / TotalElectricityInput
EHP_EleCombinedCycle = ElectricityCombinedCycle / TotalElectricityInput
EHP_EleGasTurbine = ElectricityGasTurbine / TotalElectricityInput
EHP_EleWtE = ElectricityWtE / TotalElectricityInput
EHP_EleORC = ElectricityORC / TotalElectricityInput

In [15]:
EHP_usephaseelectricity = (1 / SCOP) * EnergyProduced * RSP * EHP_EleGrid * Grid
EHP_usephaseelectricityPV = (1 / SCOP) * EnergyProduced * RSP * EHP_ElePV

EHP_usephaseelectricityRECSPV = EHP_usephaseelectricity * 0.0056 * RECSPV
EHP_usephaseelectricityRECSHydro = EHP_usephaseelectricity * 0.0056 * RECSHydro
EHP_usephaseelectricityRECSEolic = EHP_usephaseelectricity * 0.0056 * RECSEolic

EHP_usephaserefrigerantgas = 0.04 * massrefrigerant * RSP
EHP_usephaseethanepentafluoro_methanedifluoro = np.where(refrigerantgas == 'R290', np.zeros(len(Power)), 0.04 * massrefrigerant * RSP)

## 3.8 Module C2

In [16]:
EHP_Transport_to_treatment_plant = 50 * (overallmass/1000)

## 3.9 Module C3

In [17]:
EHP_EOL_reinforcingsteel = EHP_reinforcingsteel
EHP_EOL_lowalloyedsteel = EHP_lowalloyedsteel
EHP_EOL_copper = EHP_copper
EHP_EOL_elastomer = EHP_elastomer
EHP_EOL_PVC = EHP_PVC
EHP_EOL_airfun = EHP_airfun
EHP_EOL_electronics = EHP_electronics
EHP_EOL_lubricatingoil = EHP_lubricatingoil
EHP_EOL_refrigerantgas = massrefrigerant
EHP_EOL_steelwell =  EHP_steelwell
EHP_EOL_PEwell = EHP_PEwell
EHP_EOL_bentonitewell = EHP_bentonitewell
EHP_EOL_cementwell = EHP_cementwell
EHP_EOL_glycolethylenewell = EHP_glycolethylenewell
EHP_EOL_ironwell = EHP_ironwell

## 3.10 Module C4

In [18]:
EHP_EOL_landfill_reinforcingsteel = EHP_EOL_reinforcingsteel * LandfillMetals
EHP_EOL_landfill_lowalloyedsteel = EHP_EOL_lowalloyedsteel * LandfillMetals
EHP_EOL_landfill_copper = EHP_EOL_copper * LandfillMetals
EHP_EOL_inc_elastomer = EHP_EOL_elastomer * IncinerationElastomer
EHP_EOL_landfill_elastomer = EHP_EOL_elastomer * LandfillElastomer
EHP_EOL_inc_PVC = EHP_EOL_PVC * IncinerationPVC
EHP_EOL_landfill_PVC = EHP_EOL_PVC * LandfillPVC
EHP_EOL_landfill_airfun = EHP_EOL_airfun * LandfillMetals
EHP_EOL_inc_electronics = EHP_EOL_electronics * IncinerationElectronics #kg
EHP_EOL_landfill_electronics = EHP_EOL_electronics * LandfillElectronics #kg
EHP_EOL_inc_lubricatingoil = EHP_EOL_lubricatingoil * IncinerationLubricatingOil #kg
EHP_EOL_landfill_refrigerantgas = EHP_EOL_refrigerantgas * LandfillRefrigerant
EHP_EOL_landfill_steelwell = EHP_EOL_steelwell * LandfillMetals
EHP_EOL_inc_PEwell = EHP_EOL_PEwell * IncinerationPEHD
EHP_EOL_landfill_PEwell = EHP_EOL_PEwell * LandfillPEHD
EHP_EOL_inc_bentonitewell = EHP_EOL_bentonitewell * IncinerationHazardous #kg
EHP_EOL_landfill_cementwell = EHP_EOL_cementwell * LandfillConcrete
EHP_EOL_inc_glycolethylenewell = EHP_EOL_glycolethylenewell * IncinerationHazardous #kg
EHP_EOL_landfill_ironwell = EHP_EOL_ironwell * LandfillMetals

## 3.11 Module D1 (related to the export of secondary materials)

In [19]:
EHP_EOL_recycling_reinforcingsteel = (RecyclingMetals * RecyclingEfficiencyMetals * EHP_EOL_reinforcingsteel) - (RecycledContentMetals * EHP_EOL_reinforcingsteel)
EHP_EOL_primary_reinforcingsteel = ((RecycledContentMetals * EHP_EOL_reinforcingsteel) - (RecyclingMetals * RecyclingEfficiencyMetals * EHP_EOL_reinforcingsteel)) * SRmetals

EHP_EOL_recycling_lowalloyedsteel = (RecyclingMetals * RecyclingEfficiencyMetals * EHP_EOL_lowalloyedsteel) - (RecycledContentMetals * EHP_EOL_lowalloyedsteel)
EHP_EOL_primary_lowalloyedsteel = ((RecycledContentMetals * EHP_EOL_lowalloyedsteel) - (RecyclingMetals * RecyclingEfficiencyMetals * EHP_EOL_lowalloyedsteel)) * SRmetals

EHP_EOL_recycling_copper = (RecyclingMetals * RecyclingEfficiencyMetals * EHP_EOL_copper) - (RecycledContentMetals * EHP_EOL_copper)
EHP_EOL_primary_copper = ((RecycledContentMetals * EHP_EOL_copper) - (RecyclingMetals * RecyclingEfficiencyMetals * EHP_EOL_copper)) * SRmetals

EHP_EOL_recycling_airfun = (RecyclingMetals * RecyclingEfficiencyMetals * EHP_EOL_airfun) - (RecycledContentMetals * EHP_EOL_airfun)
EHP_EOL_primary_airfun = ((RecycledContentMetals * EHP_EOL_airfun) - (RecyclingMetals * RecyclingEfficiencyMetals * EHP_EOL_airfun)) * SRmetals

EHP_EOL_recycling_steelwell = (RecyclingMetals * RecyclingEfficiencyMetals * EHP_EOL_steelwell) - (RecycledContentMetals * EHP_EOL_steelwell)
EHP_EOL_primary_steelwell = ((RecycledContentMetals * EHP_EOL_steelwell) - (RecyclingMetals * RecyclingEfficiencyMetals * EHP_EOL_steelwell)) * SRmetals

EHP_EOL_recycling_PEwell = (RecyclingPEHD * RecyclingEfficiencyPlastics * EHP_EOL_PEwell) - (RecycledContentPlastic * EHP_EOL_PEwell)
EHP_EOL_primary_PEwell = ((RecycledContentPlastic * EHP_EOL_PEwell) - (RecyclingPEHD * RecyclingEfficiencyPlastics * EHP_EOL_PEwell)) * SRinert

EHP_EOL_recycling_cementwell = (RecyclingConcrete * RecyclingEfficiencyConcrete * EHP_EOL_cementwell) - (RecycledContentConcrete * EHP_EOL_cementwell)
EHP_EOL_primary_cementwell = ((RecycledContentConcrete * EHP_EOL_cementwell) - (RecyclingConcrete * RecyclingEfficiencyConcrete * EHP_EOL_cementwell)) * SRinert

EHP_EOL_recycling_ironwell = (RecyclingMetals * RecyclingEfficiencyMetals * EHP_EOL_ironwell) - (RecycledContentMetals * EHP_EOL_ironwell)
EHP_EOL_primary_ironwell = ((RecycledContentMetals * EHP_EOL_ironwell) - (RecyclingMetals * RecyclingEfficiencyMetals * EHP_EOL_ironwell)) * SRmetals

## 3.12 Module D3 (related to the export of energy as result of waste incineration)

In [20]:
EHP_EOL_energyrecoveryheat_elastomer = - EHP_EOL_inc_elastomer * LHVElastomer * Xincheat
EHP_EOL_energyrecoveryele_elastomer = - EHP_EOL_inc_elastomer * LHVElastomer * Xincele
EHP_EOL_energyrecoveryheat_PVC = - EHP_EOL_inc_PVC * LHVPVC * Xincheat
EHP_EOL_energyrecoveryele_PVC = - EHP_EOL_inc_PVC * LHVPVC * Xincele
EHP_EOL_energyrecoveryheat_electronics = - EHP_EOL_inc_electronics * LHVElectronics * Xincheat
EHP_EOL_energyrecoveryele_electronics = - EHP_EOL_inc_electronics * LHVElectronics * Xincele
EHP_EOL_energyrecoveryheat_lubricatingoil = - EHP_EOL_inc_lubricatingoil * LHVHazardous * Xincheat
EHP_EOL_energyrecoveryele_lubricatingoil = - EHP_EOL_inc_lubricatingoil * LHVHazardous * Xincele
EHP_EOL_energyrecoveryheat_PEwell = - EHP_EOL_inc_PEwell * LHVHDPE * Xincheat
EHP_EOL_energyrecoveryele_PEwell = - EHP_EOL_inc_PEwell * LHVHDPE * Xincele
EHP_EOL_energyrecoveryheat_bentonitewell = - EHP_EOL_inc_bentonitewell * LHVHazardous * Xincheat
EHP_EOL_energyrecoveryele_bentonitewell = - EHP_EOL_inc_bentonitewell * LHVHazardous * Xincele
EHP_EOL_energyrecoveryheat_glycolethylenewell = - EHP_EOL_inc_glycolethylenewell * LHVHazardous * Xincheat
EHP_EOL_energyrecoveryele_glycolethylenewell = - EHP_EOL_inc_glycolethylenewell * LHVHazardous * Xincele

# 4. Data for export

In [21]:
ThermalEnergyProduced_EHP = np.array([EnergyProduced * RSP])